In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
class UserItemDataset(Dataset):
    def __init__(self, user_item_pairs, tokenizer, max_length):
        self.user_item_pairs = user_item_pairs
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.user_item_pairs)

    def __getitem__(self, idx):
        user, item = self.user_item_pairs[idx]
        # Encode the user-item pair
        encoding = self.tokenizer.encode_plus(
            f'user {user} item {item}',
            add_special_tokens=True,
            max_length=self.max_length,
            return_tensors='pt',
            padding='max_length',
            truncation=True
        )
        return encoding['input_ids'].flatten(), encoding['attention_mask'].flatten()

In [ ]:
# Loading a pretrained LLM and tokenizer
model_name = 'gpt-2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [ ]:
# dataset and dataloader
user_item_pairs = [('user1', 'item1'), ('user2', 'item2')]
dataset = UserItemDataset(user_item_pairs, tokenizer, max_length=128)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [ ]:
# generating recommendations
def generate_recommendations(prompt, model, tokenizer, max_length):
    inputs = tokenizer.encode(prompt, return_tensors='pt')
    outputs = model.generate(inputs, max_length=max_length)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
# Example usage: Generate a recommendation for a user
user_prompt = "user user1"  # Replace with the actual user prompt
recommendation = generate_recommendations(user_prompt, model, tokenizer, max_length=128)
print(f'Recommended item: {recommendation}')